<a href="https://colab.research.google.com/github/suryawangsa/suryawangsa.github.io/blob/main/testing_klasifikasi_topeng_bali.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kagglehub
import kagglehub
import os


path = kagglehub.dataset_download("suryapradana/balinese-mask")
print("Dataset path:", path)
print("Isi folder:", os.listdir(path))


100%|██████████| 0.99G/0.99G [00:14<00:00, 71.6MB/s]

Extracting files...


Dataset path: /root/.cache/kagglehub/datasets/suryapradana/balinese-mask/versions/1
Isi folder: ['balinese mask']


In [28]:
import kagglehub
import os

# Download dataset
path = kagglehub.dataset_download("suryapradana/balinese-mask")
print("Path ke dataset:", path)

# Lihat isi folder utama
print("Isi folder utama:")
print(os.listdir(path))

# Jika perlu, lihat isi dalam subfolder
for item in os.listdir(path):
    item_path = os.path.join(path, item)
    if os.path.isdir(item_path):
        print(f"Isi folder '{item}':", os.listdir(item_path))


Path ke dataset: /kaggle/input/balinese-mask
Isi folder utama:
['balinese mask']
Isi folder 'balinese mask': ['bujuh', 'penasar', 'tua', 'wijil', 'dalem', 'sidakarya', 'keras']


In [29]:
dataset_dir = os.path.join(path, "balinese mask")
# Cek ulang
print("Folder dataset_dir:", dataset_dir)
print("Isi:", os.listdir(dataset_dir))


Folder dataset_dir: /kaggle/input/balinese-mask/balinese mask
Isi: ['bujuh', 'penasar', 'tua', 'wijil', 'dalem', 'sidakarya', 'keras']


In [31]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Path sudah benar
dataset_dir = os.path.join(path, "balinese mask")

# Output folder
base_output = "/content/dataset_split"
train_dir = os.path.join(base_output, "train")
val_dir = os.path.join(base_output, "val")

# Buat folder train dan val serta subfolder per kelas
for split_dir in [train_dir, val_dir]:
    os.makedirs(split_dir, exist_ok=True)
    for class_name in os.listdir(dataset_dir):
        os.makedirs(os.path.join(split_dir, class_name), exist_ok=True)

# Bagi data
for class_name in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_name)

    # Ambil hanya file (hindari folder seperti 'bak')
    images = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]

    train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

    for img in train_imgs:
        shutil.copy2(os.path.join(class_path, img), os.path.join(train_dir, class_name, img))

    for img in val_imgs:
        shutil.copy2(os.path.join(class_path, img), os.path.join(val_dir, class_name, img))

print("✅ Dataset berhasil dibagi menjadi train dan val.")


✅ Dataset berhasil dibagi menjadi train dan val.


In [32]:
import tensorflow as tf

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/dataset_split/train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/dataset_split/val",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# Percepat loading
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)


Found 318 files belonging to 8 classes.
Found 104 files belonging to 8 classes.


In [35]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

# Load base MobileNetV2 tanpa output layer
base_model = MobileNetV2(input_shape=(224, 224, 3),
                         include_top=False,
                         weights='imagenet')
base_model.trainable = False  # freeze dulu
raw_train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/dataset_split/train",
    image_size=(224, 224),
    batch_size=32
)
class_names = raw_train_ds.class_names

train_ds = raw_train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

# Build model klasifikasi
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(class_names), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Found 318 files belonging to 8 classes.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,984 (9.24 MB)

 Trainable params: 165,000 (644.53 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [36]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10  # bisa kamu sesuaikan
)


Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 55s 5s/step - accuracy: 0.2171 - loss: 2.2577 - val_accuracy: 0.5673 - val_loss: 1.2942
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 79s 4s/step - accuracy: 0.5536 - loss: 1.3072 - val_accuracy: 0.7308 - val_loss: 0.9078
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 92s 6s/step - accuracy: 0.6960 - loss: 0.8841 - val_accuracy: 0.9231 - val_loss: 0.5097
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 80s 6s/step - accuracy: 0.8734 - loss: 0.5677 - val_accuracy: 0.8942 - val_loss: 0.4006
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 78s 5s/step - accuracy: 0.8808 - loss: 0.4495 - val_accuracy: 0.9231 - val_loss: 0.3824
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 55s 6s/step - accuracy: 0.9109 - loss: 0.3547 - val_accuracy: 0.9327 - val_loss: 0.2772
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 81s 5s/step - accuracy: 0.9039 - loss: 0.3041 - val_accuracy: 0.9615 - val_loss: 0.1848
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 54s 6s/step - accuracy: 0.9665 - loss: 0.2172 - val_accuracy: 0.9423 - val_loss:

In [41]:
import os

val_tua_path = "/content/dataset_split/val/tua"
print("Isi folder val/tua:")
print(os.listdir(val_tua_path))

Isi folder val/tua:
['173.jpg', '170.JPG', '161.jpg', '159.jpg', '168.JPG', '171.jpg', '177.JPG', '191.jpg', '162.jpg', '180.JPG', '160.jpg', '172.jpg', '192.JPG', '197.jpg', '184.jpg', '193.JPG', '185.jpg', '190.jpg', '169.JPG', '187.jpg']


In [42]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Contoh prediksi 1 gambar
img_path = "/content/dataset_split/val/tua/173.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

predictions = model.predict(img_array)
predicted_class = class_names[np.argmax(predictions)]

print("Prediksi:", predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Prediksi: tua


In [43]:
model.save("mobilenetv2_topeng_bali.h5")

In [45]:
import os

val_keras_path = "/content/dataset_split/val/keras"
print("Isi folder val/keras:")
print(os.listdir(val_keras_path))

Isi folder val/keras:
['101.jpg', '104.jpg', '140.JPG', '129.JPG', '142.jpg', '146.JPG', '100.JPG', '136.jpg', '111.jpg', '125.jpg']


In [46]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Contoh prediksi 1 gambar
img_path = "/content/dataset_split/val/keras/101.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

predictions = model.predict(img_array)
predicted_class = class_names[np.argmax(predictions)]

print("Prediksi:", predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
Prediksi: keras
